In [1]:
import cv2
import dlib
import numpy as np
import time
from collections import deque

# Load dlib's pre-trained face detector and facial landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Get facial landmark indices for the eyes
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

# Microsaccade parameters
MICROSACCADE_VELOCITY_THRESHOLD = 20  # Velocity threshold in pixels/second
SUSTAINED_TIME_THRESHOLD = 5  # Time in seconds
FRAME_WINDOW_SIZE = 5  # Number of frames in the 5-second window
FRAME_THRESHOLD = 0.8  # 80% of frames should exceed the threshold

# Function to calculate midpoint between two points
def midpoint(point1, point2):
    return int((point1.x + point2.x) / 2), int((point1.y + point2.y) / 2)

# Function to determine the gaze direction (screen or outside)
def get_gaze_ratio(eye_points, facial_landmarks, frame):
    eye_region = np.array([(facial_landmarks.part(point).x, facial_landmarks.part(point).y) for point in eye_points])
    height, width, _ = frame.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.polylines(mask, [eye_region], isClosed=True, color=255, thickness=2)
    cv2.fillPoly(mask, [eye_region], color=255)

    eye_frame = cv2.bitwise_and(frame, frame, mask=mask)
    min_x = np.min(eye_region[:, 0])
    max_x = np.max(eye_region[:, 0])
    min_y = np.min(eye_region[:, 1])
    max_y = np.max(eye_region[:, 1])

    gray_eye = cv2.cvtColor(eye_frame[min_y:max_y, min_x:max_x], cv2.COLOR_BGR2GRAY)
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(threshold_eye, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(max_contour)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])  # Centroid of the contour (pupil)
            cy = int(M["m01"] / M["m00"])

            cv2.circle(eye_frame, (min_x + cx, min_y + cy), 4, (0, 0, 255), 2)

            eye_width = max_x - min_x
            gaze_ratio = cx / eye_width  # Ratio of where the pupil is located

            return gaze_ratio
    return None

# Function to determine pupil position
def get_pupil_position(eye_points, facial_landmarks, frame):
    eye_region = np.array([(facial_landmarks.part(point).x, facial_landmarks.part(point).y) for point in eye_points])

    height, width, _ = frame.shape
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.polylines(mask, [eye_region], isClosed=True, color=255, thickness=2)
    cv2.fillPoly(mask, [eye_region], color=255)

    eye_frame = cv2.bitwise_and(frame, frame, mask=mask)
    min_x = np.min(eye_region[:, 0])
    max_x = np.max(eye_region[:, 0])
    min_y = np.min(eye_region[:, 1])
    max_y = np.max(eye_region[:, 1])

    gray_eye = cv2.cvtColor(eye_frame[min_y:max_y, min_x:max_x], cv2.COLOR_BGR2GRAY)
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(threshold_eye, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(max_contour)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])  # Centroid of the contour (pupil)
            cy = int(M["m01"] / M["m00"])

            cv2.circle(eye_frame, (min_x + cx, min_y + cy), 4, (0, 0, 255), 2)
            return (min_x + cx, min_y + cy)  # Return the absolute position of the pupil

    return None

# Main function to detect both microsaccades and inattention
def detect_microsaccades_and_inattention(video_source=1):
    cap = cv2.VideoCapture(video_source)

    prev_left_pupil = None
    prev_right_pupil = None
    prev_time = time.time()

    left_velocity_window = deque(maxlen=FRAME_WINDOW_SIZE)
    right_velocity_window = deque(maxlen=FRAME_WINDOW_SIZE)

    # Variables to track inattention time
    inattention_frames = 0
    total_frames = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)

            # Microsaccade detection logic
            left_pupil = get_pupil_position(LEFT_EYE_POINTS, landmarks, frame)
            right_pupil = get_pupil_position(RIGHT_EYE_POINTS, landmarks, frame)
            current_time = time.time()
            time_elapsed = current_time - prev_time

            if left_pupil and right_pupil and time_elapsed > 0:
                if prev_left_pupil and prev_right_pupil:
                    left_movement = np.linalg.norm(np.array(left_pupil) - np.array(prev_left_pupil))
                    right_movement = np.linalg.norm(np.array(right_pupil) - np.array(prev_right_pupil))

                    left_velocity = left_movement / time_elapsed
                    right_velocity = right_movement / time_elapsed

                    left_velocity_window.append(left_velocity)
                    right_velocity_window.append(right_velocity)

                    left_above_threshold = sum(1 for v in left_velocity_window if v > MICROSACCADE_VELOCITY_THRESHOLD) / len(left_velocity_window)
                    right_above_threshold = sum(1 for v in right_velocity_window if v > MICROSACCADE_VELOCITY_THRESHOLD) / len(right_velocity_window)

                    if left_above_threshold >= FRAME_THRESHOLD and right_above_threshold >= FRAME_THRESHOLD:
                        print(f"Sustained microsaccade detected!")
                        
                        video_player.pause()
                    
                        # Show the message
                        display_message_on_player("Microsaccade detected!")

                prev_left_pupil = left_pupil
                prev_right_pupil = right_pupil
                prev_time = current_time

            # Inattention detection logic
            left_gaze_ratio = get_gaze_ratio(LEFT_EYE_POINTS, landmarks, frame)
            right_gaze_ratio = get_gaze_ratio(RIGHT_EYE_POINTS, landmarks, frame)

            if left_gaze_ratio is not None and right_gaze_ratio is not None:
                gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2.0

                if gaze_ratio < 0.4 or gaze_ratio > 0.6:
                    cv2.putText(frame, "Inattentive", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    inattention_frames += 1
                else:
                    cv2.putText(frame, "Attentive", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                total_frames += 1

                current_time = time.time()
                if current_time - start_time >= 5:
                    if (inattention_frames / total_frames) >= 0.8:
                        print("Alert: Student has been inattentive for 5 seconds!")
                        cv2.putText(frame, "ALERT: Inattentive!", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    start_time = time.time()
                    inattention_frames = 0
                    total_frames = 0

        cv2.imshow("Microsaccade and Inattention Detector", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
import json
import requests
import sys
import threading
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QLabel, QInputDialog, QMessageBox
from PyQt5.QtWebEngineWidgets import QWebEngineView, QWebEnginePage
from PyQt5.QtCore import QUrl, QTimer, pyqtSignal, QObject
import cv2
import dlib
import numpy as np
import time
from collections import deque

# Define a signal class for thread-safe communication between the detector and UI
class Communicate(QObject):
    update_signal = pyqtSignal(str)

url = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

class YouTubePlayer(QMainWindow):
    def __init__(self, video_url):
        super().__init__()

        self.setWindowTitle('YouTube Video Player')
        self.setGeometry(100, 100, 800, 600)

        # Create a widget to hold the video
        widget = QWidget(self)
        self.setCentralWidget(widget)

        # Create a layout to embed the video player
        layout = QVBoxLayout()
        widget.setLayout(layout)

        # Create a QWebEngineView to display the YouTube video
        self.browser = QWebEngineView()
        layout.addWidget(self.browser)

        # Embed the YouTube video using the video URL
        embed_url = f"https://www.youtube.com/embed/{self.extract_video_id(video_url)}"
        self.browser.setUrl(QUrl(embed_url))

        # Initialize the communicator
        self.comm = Communicate()
        self.comm.update_signal.connect(self.update_status)
        
        self.message_label = QLabel(self)
        self.message_label.setStyleSheet("background-color: red; color: white; font-size: 18px; padding: 10px;")
        self.message_label.setText("Microsaccade detected!")
        self.message_label.setVisible(False)  # Initially hidden
        layout.addWidget(self.message_label)
        
        # Start the microsaccade detection in a separate thread
        self.detection_thread = threading.Thread(target=self.detect_microsaccades_and_inattention)
        self.detection_thread.daemon = True  # Daemonize the thread so it exits with the main program
        self.detection_thread.start()

    def extract_video_id(self, url):
        """Extract the YouTube video ID from the given URL."""
        if "youtube.com" in url:
            # Extract the video ID from a youtube URL
            video_id = url.split('v=')[1].split('&')[0]
            return video_id
        elif "youtu.be" in url:
            # Extract the video ID from a youtu.be URL
            return url.split('/')[-1]
        else:
            raise ValueError("Invalid YouTube URL")

    def update_status(self, message):
        """Update the UI with the detection status (e.g., inattention)."""
        print(message)
        if message == "microsaccade":
            self.pause_video()
            self.message_label.setText("Microsaccade detected!")
            self.message_label.setVisible(True)
        elif message == "resume":
            self.resume_video()
            self.message_label.setVisible(False)
        elif message == "inattention":
            self.pause_video()
            self.prompt_summary()
            # self.message_label.setText("Inattention detected for 5 seconds!")
            # self.message_label.setVisible(True)

    def pause_video(self):
        """Inject JavaScript to pause the video."""
        self.browser.page().runJavaScript("document.querySelector('video').pause();")

    def resume_video(self):
        """Inject JavaScript to resume the video."""
        self.browser.page().runJavaScript("document.querySelector('video').play();")
        
    def prompt_summary(self):
        """Prompt the student to enter a summary of their learnings."""
        conversation_history = [
        {"role": "system", "content": "You are speaking to a student who is distracted while watching a video. You just prompted the student to enter a summary of his/her learnings. React in an interested manner and ask probing, reflective questions on the response to engage the student further."}
        ]
        
        # Initial prompt to the student
        text, ok = QInputDialog.getText(self, "Sustained Inattention", 
                                        "You've been inattentive for a while.\nPlease summarize what you've learned so far:")

        if ok and text:
            count = 1
            while count < 4:
                conversation_history.append({"role": "user", "content": text})
                data = {
                "model": "gpt-4",
                "messages": conversation_history,
                "temperature": 0.7
                }
                response = requests.post(url, headers=headers, data=json.dumps(data))
                chatbot_reply = response.json().get("choices")[0]["message"]["content"]
                conversation_history.append({"role": "system", "content": chatbot_reply})
                text, ok = QInputDialog.getText(self,"Sustained Inattention",chatbot_reply)
                count += 1
                # # Display a message box with the entered summary
                # QMessageBox.information(self, "Summary", f"Your summary:\n{text}")
            # You can store the summary or process it further here
        else:
            QMessageBox.warning(self, "No Summary", "Please pay attention to the video.")
        
        temp = conversation_history[-1]["content"] + "Now summarize the entire conversation in a brief paragraph and provide encouragement to the student."
        conversation_history[-1]['content'] = temp
        data = {
                "model": "gpt-4",
                "messages": conversation_history,
                "temperature": 0.7
                }
        response = requests.post(url, headers=headers, data=json.dumps(data))
        summary = response.json().get("choices")[0]["message"]["content"]
        QMessageBox.information(self, "Summary", f"Your summary:\n{summary}")
            
    def detect_microsaccades_and_inattention(self, video_source=1):
        cap = cv2.VideoCapture(video_source)

        prev_left_pupil = None
        prev_right_pupil = None
        prev_time = time.time()

        left_velocity_window = deque(maxlen=5)
        right_velocity_window = deque(maxlen=5)

        # Variables to track inattention time
        inattention_frames = 0
        total_frames = 0
        start_time = time.time()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            detector = dlib.get_frontal_face_detector()
            faces = detector(gray)
            
            for face in faces:
                landmarks = predictor(gray, face)

                # Microsaccade detection logic
                left_pupil = get_pupil_position(LEFT_EYE_POINTS, landmarks, frame)
                right_pupil = get_pupil_position(RIGHT_EYE_POINTS, landmarks, frame)
                current_time = time.time()
                time_elapsed = current_time - prev_time
    
                if left_pupil and right_pupil and time_elapsed > 0:
                    if prev_left_pupil and prev_right_pupil:
                        left_movement = np.linalg.norm(np.array(left_pupil) - np.array(prev_left_pupil))
                        right_movement = np.linalg.norm(np.array(right_pupil) - np.array(prev_right_pupil))
    
                        left_velocity = left_movement / time_elapsed
                        right_velocity = right_movement / time_elapsed
    
                        left_velocity_window.append(left_velocity)
                        right_velocity_window.append(right_velocity)
    
                        left_above_threshold = sum(1 for v in left_velocity_window if v > MICROSACCADE_VELOCITY_THRESHOLD) / len(left_velocity_window)
                        right_above_threshold = sum(1 for v in right_velocity_window if v > MICROSACCADE_VELOCITY_THRESHOLD) / len(right_velocity_window)
    
                        if left_above_threshold >= FRAME_THRESHOLD and right_above_threshold >= FRAME_THRESHOLD:
                            print(f"Sustained microsaccade detected!")
                            self.comm.update_signal.emit("microsaccade")
                            time.sleep(3)  # Pause for 3 seconds
                            self.comm.update_signal.emit("resume")
    
                    prev_left_pupil = left_pupil
                    prev_right_pupil = right_pupil
                    prev_time = current_time
    
                # Inattention detection logic
                left_gaze_ratio = get_gaze_ratio(LEFT_EYE_POINTS, landmarks, frame)
                right_gaze_ratio = get_gaze_ratio(RIGHT_EYE_POINTS, landmarks, frame)
    
                if left_gaze_ratio is not None and right_gaze_ratio is not None:
                    gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2.0
    
                    if gaze_ratio < 0.4 or gaze_ratio > 0.6:
                        cv2.putText(frame, "Inattentive", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        inattention_frames += 1
                    else:
                        cv2.putText(frame, "Attentive", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
                    total_frames += 1
    
                    current_time = time.time()
                    if current_time - start_time >= 5:
                        if (inattention_frames / total_frames) >= 0.8:
                            print("Alert: Student has been inattentive for 5 seconds!")
                            self.comm.update_signal.emit("inattention")
                            time.sleep(3)  # Pause for 3 seconds
                            self.comm.update_signal.emit("resume")
    
                        start_time = time.time()
                        inattention_frames = 0
                        total_frames = 0

            time.sleep(0.1)  # Optional: Add a small delay to prevent maxing out CPU

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    video_url = "https://www.youtube.com/watch?v=FWTNMzK9vG4"
    window = YouTubePlayer(video_url)
    window.show()
    sys.exit(app.exec_())

/Users/PrakharGoel/Desktop/hackathon/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
